In [1]:
#%matplotlib notebook

import time
import pandas as pd
import numpy as np

import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
#from pyemma import msm # not available on Kaggle Kernel
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM


In [2]:
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        #distance.set_value(i, np.linalg.norm(Xa-Xb))
        distance.at[i] = np.linalg.norm(Xa-Xb)
    return distance

# train markov model to get transition matrix
def getTransitionMatrix (df):
    df = np.array(df)
    model = msm.estimate_markov_model(df, 1)
    return model.transition_matrix

def markovAnomaly(df, windows_size, threshold):
    transition_matrix = getTransitionMatrix(df)
    real_threshold = threshold**windows_size
    df_anomaly = []
    for j in range(0, len(df)):
        if (j < windows_size):
            df_anomaly.append(0)
        else:
            sequence = df[j-windows_size:j]
            sequence = sequence.reset_index(drop=True)
            df_anomaly.append(anomalyElement(sequence, real_threshold, transition_matrix))
    return df_anomaly

In [17]:
## CSV : convert=> UTC -> [DATE time] cpu_value->[cpu]
df = pd.read_csv('../cto_k8s/m_data_10.11.1.80:9091.csv', usecols=['time', 'cpu_value'])
df.rename(columns={'cpu_value':'cpu'}, inplace=True)

df['timestamp'] = pd.to_datetime(df.time, unit='s')
df = df[['timestamp', 'cpu']]
df['cpu'] = df['cpu'].fillna(0)

df.head()




,timestamp,cpu
0,2020-08-04 00:00:00,0.565333
1,2020-08-04 00:10:00,0.571564
2,2020-08-04 00:20:00,0.543800
3,2020-08-04 00:30:00,0.534930
4,2020-08-04 00:40:00,0.569663


In [18]:
df.tail()

,timestamp,cpu
3894,2020-08-31 03:20:00,3.012313
3895,2020-08-31 03:30:00,2.939557
3896,2020-08-31 03:40:00,2.004364
3897,2020-08-25 05:10:00,0.000000
3898,2020-08-25 05:30:00,0.000000
